In [ ]:
import sys
sys.path.append("..")
from common_utils import set_data_home
set_data_home("~/datasets")
from common_utils import DATA_HOME, join
from lstm.sales_data import Sales_Dataset
from torch.utils.data.dataloader import DataLoader
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
SALE_HOME = join(DATA_HOME, "sales_data")
MODEL_HOME = join(DATA_HOME, "sale_model")

In [ ]:
from torch.nn import LSTM

I, H, B = 71, 34, 3

sd = Sales_Dataset(SALE_HOME, seq_len=500, is_train=False, device=device)

model = LSTM(I, H, batch_first=True).cuda()
# model.load_state_dict(torch.load("./sales_model_600.pth"))
sd[0][1].shape, sd[1][1].shape, sd[2][1].shape

In [ ]:
import pandas as pd
ans_df = pd.DataFrame()
for X, y, store_id, date in sd:
    yhat, (_, _) = model(X.cuda())
    sd.set_log_rets(yhat[-1, :], store_id, date)
    print(f"store_id: {store_id}, date: {date} infered")

sd.TR.to_csv("./TR.csv", index=False)

### Transform returns to sales

In [ ]:
from datetime import timedelta
import pandas as pd
sales = pd.read_csv(join(SALE_HOME, "test.csv"), index_col=False)
sales["sales"] = 0.
sales.sort_values(["store_nbr", "family", "date"], inplace=True)
# sd.TR = pd.read_csv(join(SALE_HOME, "TR.csv"), index_col=False)
sd.TR = sd.TR.reset_index().sort_values(["store_nbr", "family", "date"])
base_sales = sd.base_sales

for s in sd.ids:
    for f in sd.families:
        start_date, end_date = sd.train_max_date + timedelta(days=1), sd.test_max_date
        prev_sale = base_sales.loc[(base_sales.store_nbr == s) & (base_sales.family == f)].sales
        for d in pd.date_range(start_date, end_date):
            ret = sd.TR.loc[(sd.TR.store_nbr == s) &\
                            (sd.TR.family == f) & \
                            (sd.TR.date == d)].sales
            curr_sale = 10 ** ret.item() * prev_sale.item()
            sales.loc[(sales.store_nbr == s) & (sales.family == f) & (sales.date == d.strftime("%Y-%m-%d")), "sales"] = curr_sale
        print(f"store {s}, family {f} inferred")
            

base_sales, sd.test_max_date - sd.train_max_date, sale

### output the answer

In [ ]:
sales.drop(columns=["store_nbr", "date", "family", "onpromotion"]).sort_values("id").to_csv("answer.csv", index=False)